In [ ]:
# output 디렉토리 초기화 - 없으면 생성, 있으면 내부 jpg 파일 삭제

import os

if os.path.exists("./output"):
    listdir = os.listdir("./output")
    for f in listdir:
        if f.endswith(".jpg"):
            os.remove(f"./output/{f}")
else:
    os.makedirs("./output")

if os.path.exists("./output.zip"):
    os.remove("./output.zip")


In [ ]:
# 카메라 입력 받아온 후 녹화

try:
    import jchm
except ModuleNotFoundError:
    pass
import cv2
import os
import time
from contextlib import suppress
from enum import IntEnum
from typing import Callable

class CameraMode(IntEnum):
    JAJUCHA = 0 # 자주차
    COMPUTER = 1 # 컴퓨터

class Camera:
    def __init__(self, mode: CameraMode, device):
        if not isinstance(mode, CameraMode):
            raise TypeError(f"mode는 CameraMode여야 합니다. got {type(mode).__name__}")
        self.mode = mode

        try:
            if mode is CameraMode.JAJUCHA: # 자주차 카메라
                try:
                    self.camera = jchm.camera
                except NameError as e: # jchm가 정의되지 않았다면 (임포트되지 않았다면) NameError
                    raise ImportError("jchm 모듈을 사용할 수 없습니다.") from e
                self.set_camera_device(device)
                self.getFrame: Callable[[], "any"] = self.__getJajuchaFrame
                self.showFrame: Callable[["any"], None] = self.__showFrameOnJajucha

            elif mode is CameraMode.COMPUTER:
                self.set_camera_device(device)
                if not self.camera.isOpened():
                    raise RuntimeError(f"카메라를 인식할 수 없습니다. (index {self.device})")
                self.getFrame = self.__getComputerFrame
                self.showFrame = self.__showFrameOnComputer

            else:
                raise ValueError(f"사용불가한 mode: {mode}")

        except Exception as e: # init 전반의 예외를 포장
            raise RuntimeError("카메라 초기화 도중 예기치 않은 오류가 발생했습니다.") from e

    # --- public helpers ---

    def set_camera_device(self, new_device) -> None:
        """COMPUTER 모드에서 카메라 인덱스를 런타임 도중에 변경."""
        if self.mode is CameraMode.JAJUCHA:
            if type(new_device) is not str:
                raise TypeError("자주차 카메라의 디바이스 이름은 문자열이여야 합니다.")
            self.camera = jchm.camera
        elif self.mode is CameraMode.COMPUTER:
            if type(new_device) is not int:
                raise TypeError("컴퓨터 카메라의 디바이스 이름은 정수여야 합니다.")
            if hasattr(self, "camera") and self.camera is not None:
                self.release()
            self.device = new_device
            self.camera = cv2.VideoCapture(new_device)

    def release(self) -> None:
        """리소스 정리."""
        if self.mode is CameraMode.COMPUTER and hasattr(self, "camera") and self.camera is not None:
            self.camera.release()
            # 창 사용했다면 닫기
            try:
                cv2.destroyAllWindows()
            except Exception:
                pass
    
    def saveFrame(self, mat, output_dir: str, filename: str, jpg_compress_quality: int):
        if not os.path.exists(output_dir):
            raise NotADirectoryError("output 디렉토리를 찾지 못했습니다.")
        cv2.imwrite(f"{output_dir}/{filename}", mat, [cv2.IMWRITE_JPEG_QUALITY, jpg_compress_quality])

    # --- JAJUCHA backend ---

    def __getJajuchaFrame(self):
        return self.camera.get_image(self.device)

    def __showFrameOnJajucha(self, mat):
        self.camera.show_image(mat, "center")

    # --- COMPUTER backend ---

    def __getComputerFrame(self):
        ret, frame = self.camera.read()
        if not ret:
            raise RuntimeError(f"카메라를 인식할 수 없습니다. (index {self.device})")
        return frame

    def __showFrameOnComputer(self, mat):
        cv2.imshow("center", mat)

def main():
    OUTPUT_DIR = "./output"
    FPS = 1
    DT = 1.0 / FPS
    LOCAL_CAMERA_INDEX = 0
    JPG_COMPRESS_QUALITY = 80

    camera = Camera(
        mode=CameraMode.COMPUTER,
        device=LOCAL_CAMERA_INDEX
    )

    with suppress(KeyboardInterrupt):
        cnt = 0
        next_frame = time.perf_counter()
        while True:
            next_frame += DT
            now = time.perf_counter()
            if now < next_frame:
                time.sleep(next_frame - now)
            else:
                missed = int((now - next_frame) // DT) + 1
                next_frame += missed * DT
            cnt += 1
            img = camera.getFrame()
            camera.showFrame(img)
            camera.saveFrame(img, OUTPUT_DIR, f"{cnt}.jpg", JPG_COMPRESS_QUALITY)

if __name__ == "__main__":
    main()



In [ ]:
# output 디렉토리 압축

import shutil

!cd output && find . -exec touch -t 200001010000 {} \;

shutil.make_archive("./output", "zip", "./output/")


In [ ]:
# 파일 HTTP 정적 서빙
# 동글 연결된 디바이스에서 127.17.0.1로 연결

from http.server import ThreadingHTTPServer, SimpleHTTPRequestHandler
from functools import partial

PORT = 8000
BIND = "0.0.0.0"
DIR = os.getcwd()

def main():
    handler_cls = partial(SimpleHTTPRequestHandler, directory=DIR)
    with ThreadingHTTPServer((BIND, PORT), handler_cls) as httpd:
        root = os.path.abspath(DIR)
        print(f"Serving {root} at http://{BIND}:{PORT}")
        try:
            httpd.serve_forever()
        except KeyboardInterrupt:
            pass

if __name__ == "__main__":
    main()
